### Borena Kunze 

# Assignment One

### Library - part one

#### Importing libraries and reading the csv files

"A small college library run by volunteers has donated its books to the case study library. They utilised a simple but effective spreadsheet-based cataloguing and loan system. Their books and data files are thus available for some experimental work – although they have not been able to provide member information for data privacy reasons. The library had a standard loan period of 14 days."

We were provided with two csv files to analyse and create a scalable software system which can be applied to the entire dataset of the library.  
The first csv file contains information on 120 books, encoded as UTF-8, each with a given unique number.
The second csv file holds information on book loans structured in four rows, containing the unique number of the book loaned, the unique number of the library member, the date in which the book was loaned and the date the book has been returned, with the value 0 where the book has not been returned as of yet.  


In [31]:
#importing libraries

import csv
import datetime


#### Reading books.csv file

Storing the information into a dictionary will allow to use the unique book numbers as keys and store the information about the book as its value, making it possible for me to maintain the structure of the data.  
Keeping the book specific information as a dictionary within the dictionary, with 'Title', 'Author', 'Genre', 'Subgenre' and 'Publisher' as keys storing individual details will allow for easier iteration later on. 


In [33]:
'''
Reading the csv file into a dictionary
Key is the unique number of each book 
Value contains information about the book stored as a sub dictionary with: 
Title, Author, Genre, Subgenre, Publisher as keys and the specific information as values
Using try, except, finally for error handling
'''

try:
    with open('books.csv', mode = 'r', encoding = 'utf-8') as file:      #using 'utf-8' to encode the file
        csvFile = csv.reader(file)
        #print(csvFile)
        
        rowCount = 0
        bookDictionary = {}
        
        for row in csvFile:
            #print(row)
            if rowCount == 0:
                #reads the heading
                heading = row        
                #print(heading)
                
                rowCount += 1
            else:
                #read and store the content in the dictionary
                bookDictionary[row[0]] = { 
                                          heading[1]:row[1],
                                          heading[2]:row[2],
                                          heading[3]:row[3],
                                          heading[4]:row[4],
                                          heading[5]:row[5],
                                         }
                rowCount += 1
        print(bookDictionary)
            
except:
    print('File could not be opened.')
    
finally:
    file.close()

{'1': {'Title': 'Fundamentals of Wavelets', 'Author': 'Goswami, Jaideva', 'Genre': 'tech', 'SubGenre': 'signal_processing', 'Publisher': 'Wiley'}, '2': {'Title': 'Data Smart', 'Author': 'Foreman, John', 'Genre': 'tech', 'SubGenre': 'data_science', 'Publisher': 'Wiley'}, '3': {'Title': 'God Created the Integers', 'Author': 'Hawking, Stephen', 'Genre': 'tech', 'SubGenre': 'mathematics', 'Publisher': 'Penguin'}, '4': {'Title': 'Superfreakonomics', 'Author': 'Dubner, Stephen', 'Genre': 'science', 'SubGenre': 'economics', 'Publisher': 'HarperCollins'}, '5': {'Title': 'Orientalism', 'Author': 'Said, Edward', 'Genre': 'nonfiction', 'SubGenre': 'history', 'Publisher': 'Penguin'}, '6': {'Title': 'Nature of Statistical Learning Theory, The', 'Author': 'Vapnik, Vladimir', 'Genre': 'tech', 'SubGenre': 'data_science', 'Publisher': 'Springer'}, '7': {'Title': 'Integration of the Indian States', 'Author': 'Menon, V P', 'Genre': 'nonfiction', 'SubGenre': 'history', 'Publisher': 'Orient Blackswan'}, '8

With the reading of the csv file it becomes evident that there are values missing, such as Authors and Publisher. An example of this would be on key:  
'119': {'Title': 'Karl Marx Biography', 'Author': '', 'Genre': 'nonfiction', 'SubGenre': 'autobiography', 'Publisher': ''}  
There might be different explanations why this data is missing. As for the publisher one possible explanation might be that the books are very old and the publishing house is no longer in operation, or they have been self published, which in both cases would result in missing information about the publisher.  
With regards to the missing authors the book on key 119 is defined as an 'autobiography' in the subgenre, which implies the author's name. In other cases various authors might have contributed, which is difficult to verify by looking only at the provided date.  
With this in mind I have decided to keep these entries and assign the missing values as 'unknown' in the further steps, keeping enough data for further analysis.  
As all entries, including numbers are stored as strings, numbers might have to be turned into integers at a later point to allow for analysis and further calculations.

#### Reading the bookloans.csv file and Preparation

As the dates are stored in Microsoft epoch format, they firstly will need to be converted into a readable format.  The formatDate function will allow for the dates to be displayed as Year-Month-Date as they are read from the csv file.   
In this step it is practicable to exclude all entries with the unique book numbers that do not occur in the first file, as without the book titles and genres we will not be able to extract any information from these entries. These would be all book ids from 121 onwards.
At the same time entries where the books are still on loan will not provide any information regarding the average loan time and therefore can be excluded as well.
In return there will be the occurning unique book ids, the member ids as well as the loan and return dates, which  can be used to calculate average time of loan and how many times books have been loaned.  


In [37]:
#convert the numerical data to dates

def formatDate(refValue, dateValue):
    '''
    Function converts Microsoft epoch formatted dates 
    return: readable date as Year-Month-Day 
    '''
    #considering epoch date to be 1900-01-01
    formatRef = datetime.datetime.strptime(refValue, "%Y-%m-%d")
    newDate = (formatRef + datetime.timedelta(days=int(dateValue))).strftime("%Y-%m-%d")
    return newDate




'''
Opening and reading in the bookloans.csv file 
Excluding all rows where the unique book id is > 120 
Excluding the rows where the return date is 0
Using try, except, finally for error handling
'''
# open the file bookloans.csv
try:
    # open the file 
    with open('bookloans.csv', mode = 'r', encoding = 'utf-8') as file:       #using 'utf-8' to encode
        csvFile = csv.reader(file)
        #print(csvFile)

        #initialize the variables to store the data
        date_of_loan = []
        date_of_return = []
        memberId = []
        bookId = []

        #read the data
        for row in csvFile:
            #changing first element into '1'
            if row[0].isdigit() == False:           
                row[0] = '1'
                # excluding elements where return_date is 0
                # excluding elements where the bookId > 120
            if row[3] != 0 and int(row[0]) < 121:
            
                if row[0] != 0:                           #reading in rows 0 and 1
                    bookId += [row[0]]
                if row[1] != 0:
                    memberId = [row[1]]
                if row[2] != 0:
                #changing epoch into dates
                    date_of_loan += [formatDate('1900-01-01', row[2])]   #append the formatDate function
                else:
                    date_of_loan += ['Nan']
                if row[3] != 0:
                    date_of_return += [formatDate('1900-01-01', row[3])]
                else:
                    date_of_return += ['Nan']


        
except:
    print('File could not be opened.')
    
finally:
    file.close()

print(bookId)
print(memberId)
print(date_of_loan)
print(date_of_return)
#print(len(bookId))

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', '9', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '12', '12', '12', '12', '12', '12', '12', '12', '12', '12', '12', '12', '12', 

In [4]:
int(date_of_loan[0][0:4])


2019

#### Data Preparation

The following function will replace all missing values for 'Author' and 'Publisher' in bookDictionary (which was created from the first file) with the value 'unknown'.  
This will allow us to gather information on the frequency of loan from all the books which can be loaned from the library.  


In [38]:

def replaceVal(bookDictionary):
    '''
    Replaces empty values for missing authors and publishers in dictionary with "unknown"
    Returns: bookDictionary

    '''
    for key, value in bookDictionary.items():
        
        for k, v in value.items():
            
            if k == 'Author' and v == '':
                bookDictionary[key]['Author'] = 'unknown'       #replaces empty value of 'Author' with 'unknown'
            
            if k == 'Publisher' and v == '':
                bookDictionary[key]['Publisher'] = 'unknown'    #replaces empty value of 'Publisher' with 'unknown'
            
    return bookDictionary

    
bookDictionaryNew = replaceVal(bookDictionary)                 # applies replaceVal function to bookDictionary
print(bookDictionaryNew)    

{'1': {'Title': 'Fundamentals of Wavelets', 'Author': 'Goswami, Jaideva', 'Genre': 'tech', 'SubGenre': 'signal_processing', 'Publisher': 'Wiley'}, '2': {'Title': 'Data Smart', 'Author': 'Foreman, John', 'Genre': 'tech', 'SubGenre': 'data_science', 'Publisher': 'Wiley'}, '3': {'Title': 'God Created the Integers', 'Author': 'Hawking, Stephen', 'Genre': 'tech', 'SubGenre': 'mathematics', 'Publisher': 'Penguin'}, '4': {'Title': 'Superfreakonomics', 'Author': 'Dubner, Stephen', 'Genre': 'science', 'SubGenre': 'economics', 'Publisher': 'HarperCollins'}, '5': {'Title': 'Orientalism', 'Author': 'Said, Edward', 'Genre': 'nonfiction', 'SubGenre': 'history', 'Publisher': 'Penguin'}, '6': {'Title': 'Nature of Statistical Learning Theory, The', 'Author': 'Vapnik, Vladimir', 'Genre': 'tech', 'SubGenre': 'data_science', 'Publisher': 'Springer'}, '7': {'Title': 'Integration of the Indian States', 'Author': 'Menon, V P', 'Genre': 'nonfiction', 'SubGenre': 'history', 'Publisher': 'Orient Blackswan'}, '8

When comparing the dictionary now with the first print, it is evident that empty values have been replaced.   
'119': {'Title': 'Karl Marx Biography', 'Author': 'unknown', 'Genre': 'nonfiction', 'SubGenre': 'autobiography', 'Publisher': 'unknown'}  

#### Task 1

In the following step I will create a dictionary reporting on the frequency in which each book is loaned, starting with the least frequent. This will allow to understand which books are the most and which the least popular amongst the library members, giving an understanding on which books might need to be purchased again and which are remaining mostly unused.  
To allow for scalability and not produce a csv file every time the function is run, I will display the results in the notebook. 

The first step would be to extract all entries for the year 2019, as these are the entries of interest for the task showing the most recent trends within the library.  
The second step will be to count the times each book occurs, so they can be sorted into least frequent first.  
The third step is to create the dictionary with Author and Title as key and the number each book occurs as a value. As dictionary keys needs to be unique it is reasonable to build the key as a tuple containing the name of the author and the book title.  


In [40]:
'''
Extracts years 2019 from date_of_loan
appends to the empty list "index"
'''

index = []

#extract years 2019 and position 
for i in range(0,len(date_of_loan)):
    if int(date_of_loan[i][0:4]) == 2019:         #extract years 2019
        index.append(i)
        
#print(len(index), index)



'''
Extracts books loaned in 2019
Counts occurency for each book
'''

bookIdSelected = []

for element in index:
    bookIdSelected.append(bookId[element])    #extract books loaned in 2019
    
counts = []    
for i in range(1,121):
    count = bookIdSelected.count(str(i))    # counted books for each number
    counts.append(count)
    
#print(counts)


'''
Creates a new dictionary with the 
Key: Author name and title as tuple
Value: frequency of book loan
'''

#create new dict with name&title as key and number of times as values
#sort the keys
sortedDictionary = {}
for i in range(0, len(counts)):
    author = bookDictionary[str(i+1)]['Author']             #i starts from 0, (i+1) to start from 1 
    title = bookDictionary[str(i+1)]['Title']               #author and title
    sortedDictionary[(author, title)] = counts[i]           #author, title as key tuple - counts as value

#print(sortedDictionary)


'''
Sorts the keys of the dictionary based on their values in reverse order
Builds a new sorted dictionary 
Prints the sorted dictionary
'''

#sort the keys of sortedDictionary based on values
sortedKeys = sorted(sortedDictionary, key = sortedDictionary.get , reverse = False )
#print(sortedKeys)

#create new dictionary sDictionary
#go through sortedKeys and assign the coresponding value from sortedDictionary
sDictionary = {}
for key in sortedKeys:
    sDictionary[key] = sortedDictionary[key]              
    
print(sDictionary)


    

#bookIdSelected = bookId[index]
#print(len(index), len(bookIdSelected))
#print(bookIdSelected)



{('Villani, Cedric', 'Birth of a Theorem'): 1, ('Feynman, Richard', "Surely You're Joking Mr Feynman"): 2, ('Stonier, Alfred', 'Textbook of Economic Theory'): 2, ('Bradsky, Gary', 'Learning OpenCV'): 2, ('Sen, Amartya', 'Argumentative Indian, The'): 2, ('Maugham, William S', 'Ashenden of The British Agent'): 2, ('Rand, Ayn', 'Return of the Primitive'): 3, ('Woodward, Bob', "All the President's Men"): 3, ('Russell, Bertrand', 'History of Western Philosophy'): 3, ('Bach, Richard', 'One'): 3, ('Dubner, Stephen', 'Superfreakonomics'): 4, ('Nisbet, Robert', 'Data Mining Handbook'): 4, ('Steinbeck, John', 'Russian Journal, A'): 4, ('Dubner, Stephen', 'Freakonomics'): 4, ('Deshpande, Sunita', 'Ahe Manohar Tari'): 4, ('Lapierre, Dominique', 'O Jerusalem!'): 4, ('Steinbeck, John', 'Moon is Down, The'): 5, ('Nayar, Kuldip', 'Scoop!'): 5, ('Deshpande, P L', 'Radiowaril Bhashane & Shrutika'): 5, ('Doyle, Arthur Conan', 'Complete Sherlock Holmes, The - Vol I'): 6, ('Durant, Will', 'Story of Philoso

The created dictionary reports on the frequency of the books being loaned with 28 being the highest number for:  
"('Archer, Jeffery', 'False Impressions'): 28" and "('Tharoor, Shashi', 'Great Indian Novel, The'): 28"  
and 1 being the lowest for    
"('Villani, Cedric', 'Birth of a Theorem'): 1".

#### Task 2

The following task will generate a report on the most popular genres and subgenres based on the dicitonary showing the frequency of loan.  
This report will be used to reflect upon the interests of the readers and could possibly influence further library purchases.  
Again as to not produce a csv file on every step, I have decided to print the report within the notebook.

In [46]:
# most popular genres and subgenres

try:
    def popGenres():
        genres = []
        genresCount = 0
        subgenres = []
        subgenresCount = 0

        for row in sDictionary:
            genres.append(row[3])
            subgenres.append(row[4])
            for genre in genres:
                genresCount += 1
            for subgenre in subgenres:
                subgenresCount += 1

    print('Genres: ', genres, ':', genresCount)
    print('Subgenres: ', subgenres, ':', subgenresCount)

except:
    ('No popular genres or subgenres available')

Genres:  ['Genre', 'tech', 'tech', 'tech', 'science', 'nonfiction', 'tech', 'nonfiction', 'science', 'tech', 'nonfiction', 'tech', 'fiction', 'science', 'tech', 'nonfiction', 'fiction', 'tech', 'tech', 'fiction', 'philosophy', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'nonfiction', 'fiction', 'fiction', 'fiction', 'science', 'fiction', 'nonfiction', 'science', 'science', 'fiction', 'fiction', 'fiction', 'nonfiction', 'philosophy', 'fiction', 'nonfiction', 'fiction', 'science', 'science', 'philosophy', 'fiction', 'fiction', 'nonfiction', 'fiction', 'fiction', 'fiction', 'fiction', 'nonfiction', 'nonfiction', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'fiction', 'fiction', 'fiction', 'nonfiction', 'fiction', 'philosophy', 'fiction', 'nonfiction', 'fiction', 'nonfiction', 'nonfiction', 'nonfiction', 'tech', 'nonfiction', 'nonfiction', 'nonfiction', 'nonfiction', 'philosophy', 'nonfiction', 'nonfiction', 'nonfiction', 'fiction', 'nonfiction', 'fiction', 'nonficti

#### Task 3

In [ ]:
In this task I will be be producing a report showing the following:  
    - average lenth of a book being on loan
    - average late period (loan period 14 days)
    - proportion of users returning books late

In [ ]:
# average length of a book being on loan
# average late period (loan period 14 days)
# proportion of users returning books late

Hinweise:
Your code and documentation should be submitted as a Jupyter Notebook. In producing your solutions, you should consider how best to functionally decompose the system and what data structures are the most appropriate for producing an efficient solution. Functions should be tested, and you should demonstrate how you have achieved this. These design aspects of your solution will attract a maximum of 15 Marks of the 40 available. Achieving the correct functionality will attract a further maximum of 15 Marks and the remaining 10 Marks will be awarded to reflect the quality of your documentation. Your documentation should describe any ‘sanity checks’ that you have performed on the data, your opinion of the quality of the data and the extent to which it might inform the case-study project going forward. Also, describe any assumptions made and the extent to which you feel the completed tasks have met the requirements set. You need not submit the data files, but your code should function with the original data files present in the same folder as the Jupyter Notebook. You should submit a single Notebook (.ipynb) file.